### To run the cbgt network, see first the technical_testbench to install all the required packages and start the server 

In [1]:
#Auto reloading 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload 

In [2]:
#Removing indentation issues:
!autopep8 --in-place --aggressive --aggressive init_params.py
!autopep8 --in-place --aggressive --aggressive popconstruct.py
!autopep8 --in-place --aggressive --aggressive generateepochs.py
!autopep8 --in-place --aggressive --aggressive qvalues.py

In [ ]:
!python setup.py build_ext --inplace

In [122]:
#Importing scripts:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct as popconstruct
import qvalues as qval
import generateepochs as gen
import pdb

In [ ]:
from agent_timestep import timestep_mutator, multitimestep_mutator

# Create pipelines

### Reward pipeline

In [123]:
rsg = cbgt.Pipeline() #rsg is short for 'reward schedule generator'


(rsg.volatile_pattern,
 rsg.cp_idx,
 rsg.cp_indicator,
 rsg.noisy_pattern,
 rsg.t1_epochs,
 rsg.t2_epochs,
) = rsg[gen.GenRewardSchedule](
        rsg.n_trials,
        rsg.volatility,
        rsg.conflict,
        rsg.reward_mu,
        rsg.reward_std,
    ).shape(6)

### Network pipeline

In [124]:
#Defining necessary codeblocks: 

#MODIFIERS: 

#init_params.py: to modify the neuronal default values

def codeblock_modifycelldefaults(self):
    self.celldefaults = par.helper_cellparams(self.params)
    
def codeblock_modifypopspecific(self):
    self.popspecific = par.helper_popspecific(self.pops)
    
def codeblock_modifyreceptordefaults(self):
    self.receptordefaults = par.helper_receptor(self.receps)

def codeblock_modifybasestim(self):
    self.basestim = par.helper_basestim(self.base)
    
def codeblock_modifydpmndefaults(self):
    self.dpmndefaults = par.helper_dpmn(self.dpmns)
    
def codeblock_modifyd1defaults(self):
    self.d1defaults = par.helper_d1(self.d1)
    
def codeblock_modifyd2defaults(self):
    self.d2defaults = par.helper_d2(self.d2)

def codeblock_modifyactionchannels(self): 
    self.actionchannels = par.helper_actionchannels(self.channels)
    
#popconstruct.py: to modify population parameters 

def codeblock_popconstruct(self):
    self.popdata = popconstruct.helper_popconstruct(self.actionchannels, self.popspecific, self.celldefaults, self.receptordefaults, self.basestim, self.dpmndefaults, self.d1defaults, self.d2defaults)
    
def codeblock_poppathways(self): 
    self.pathways = popconstruct.helper_poppathways(self.popdata, self.newpathways) 

#init_params.py: Q-values initialization and update

def codeblock_init_Q_support_params(self):
    self.Q_support_params = qval.helper_init_Q_support_params()

def codeblock_update_Q_support_params(self,reward_val, chosen_action):
    self.Q_support_params = qval.helper_update_Q_support_params(self.Q_support_params,self.reward_val,pl.chosen_action)
    
def codeblock_init_Q_df(self):
    self.Q_df = qval.helper_init_Q_df(self.channels)
    
def codeblock_update_Q_df(self):
    self.Q_df, self.Q_support_params, self.dpmndefaults = qval.helper_update_Q_df(self.Q_df, self.Q_support_params, self.dpmndefaults,pl.trial_num)

In [125]:
pl = cbgt.Pipeline()

In [126]:
#Adding rsg pipeline to the network pipeline: 
pl.add(rsg) 

In [127]:
#to update the Q-values 
pl.trial_num = 0 #first row of Q-values df - initialization data 
pl.chosen_action = 2 #chosen action for the current trial

In [128]:
#Defining necessary function modules: 

#init_params.py: default neuronal values 
pl.celldefaults = par.helper_cellparams()
pl.popspecific = par.helper_popspecific()
pl.receptordefaults = par.helper_receptor()
pl.basestim = par.helper_basestim()
pl.dpmndefaults = par.helper_dpmn()
pl.d1defaults = par.helper_d1()
pl.d2defaults = par.helper_d2()
#pl.actionchannels = pl[par.helper_actionchannels]()

#popconstruct.py: default population parameters 
pl.popdata = pl[popconstruct.helper_popconstruct](pl.actionchannels, pl.popspecific, pl.celldefaults, pl.receptordefaults, pl.basestim, pl.dpmndefaults, pl.d1defaults, pl.d2defaults)
pl.pathways = pl[popconstruct.helper_poppathways](pl.popdata)

#popconstruct.py: to create connectivity grids
pl.connectivity_AMPA, pl.meaneff_AMPA, pl.plastic_AMPA = pl[popconstruct.helper_connectivity]('AMPA', pl.popdata, pl.pathways).shape(3)
pl.connectivity_GABA, pl.meaneff_GABA, pl.plastic_GABA = pl[popconstruct.helper_connectivity]('GABA', pl.popdata, pl.pathways).shape(3)
pl.connectivity_NMDA, pl.meaneff_NMDA, pl.plastic_NMDA = pl[popconstruct.helper_connectivity]('NMDA', pl.popdata, pl.pathways).shape(3)

In [129]:
#Adding codeblocks to the newtork pipeline: 
pl.add(codeblock_modifycelldefaults)
pl.add(codeblock_modifypopspecific)
pl.add(codeblock_modifyreceptordefaults)
pl.add(codeblock_modifybasestim)
pl.add(codeblock_modifydpmndefaults)
pl.add(codeblock_modifyd1defaults)
pl.add(codeblock_modifyd2defaults)
pl.add(codeblock_modifyactionchannels)
pl.add(codeblock_popconstruct)
pl.add(codeblock_poppathways)

### Q-values pipeline

In [11]:
#Creating a new pipeline for Q-values: 
q_val_pipe = cbgt.Pipeline()

In [12]:
#Defining necessary function modules: 

#qvalues.py
q_val_pipe.Q_support_params = q_val_pipe[qval.helper_init_Q_support_params]()
q_val_pipe.Q_df = q_val_pipe[qval.helper_init_Q_df](pl.actionchannels)

rsg.reward_val = q_val_pipe[qval.get_reward_value](rsg.t1_epochs,rsg.t2_epochs,pl.chosen_action,pl.trial_num) 
q_val_pipe.Q_support_params = q_val_pipe[qval.helper_update_Q_support_params](q_val_pipe.Q_support_params,rsg.reward_val,pl.chosen_action)
(q_val_pipe.Q_df, q_val_pipe.Q_support_params, pl.dpmndefaults) = q_val_pipe[qval.helper_update_Q_df](q_val_pipe.Q_df,q_val_pipe.Q_support_params,pl.dpmndefaults,pl.trial_num).shape(3)

In [13]:
#Adding the q_val_pipe to the main network pipeline pl:
pl.add(q_val_pipe)

# Running the pipeline

### 1. Default values

In [130]:
#See init_params.py / generateepochs.py / popconstruct.py / qvalues.py
#to know the parameters contained in each of the following variables 
environment = {
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : None, #action channels related parameters (init_params.py)
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df':None, #initialized Q-values df (qvalues.py)  
    'n_trials': 5, #number of trials (generateepochs.py)
    'volatility': 2, #frequency of changepoints (generateepochs.py)
    'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [131]:
locals().update(results)

In [132]:
pathways

,src,dest,receptor,type,con,eff,plastic,biselector
0,LIP`,D1STR`,AMPA`,syn`,1.0`,0.027`,True`,<function MatchBiselector.<locals>.Biselector ...
1,LIP`,D1STR`,NMDA`,syn`,1.0`,0.027`,False`,<function MatchBiselector.<locals>.Biselector ...
2,LIP`,D2STR`,AMPA`,syn`,1.0`,0.027`,True`,<function MatchBiselector.<locals>.Biselector ...
3,LIP`,D2STR`,NMDA`,syn`,1.0`,0.027`,False`,<function MatchBiselector.<locals>.Biselector ...
4,LIP`,FSI`,AMPA`,all`,1.0`,0.198`,False`,<function MatchBiselector.<locals>.Biselector ...
5,LIP`,Th`,AMPA`,all`,1.0`,0.035`,False`,<function MatchBiselector.<locals>.Biselector ...
6,LIP`,Th`,NMDA`,all`,1.0`,0.035`,False`,<function MatchBiselector.<locals>.Biselector ...
7,D1STR`,D1STR`,GABA`,syn`,0.45`,0.28`,False`,<function MatchBiselector.<locals>.Biselector ...
8,D1STR`,D2STR`,GABA`,syn`,0.45`,0.28`,False`,<function MatchBiselector.<locals>.Biselector ...
9,D1STR`,GPi`,GABA`,syn`,1.0`,2.09`,False`,<function MatchBiselector.<locals>.Biselector ...


In [133]:
popdata

,name,action,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,...,dpmn_DAp,dpmn_APRE,dpmn_APOST,dpmn_XPRE,dpmn_XPOST,dpmn_type,dpmn_alphaw,dpmn_a,dpmn_b,dpmn_c
0,GPi`,1`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
1,GPi`,2`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
2,STNE`,1`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
3,STNE`,2`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
4,GPeP`,1`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
5,GPeP`,2`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
6,D1STR`,1`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,1`,18.333333333333332`,1.0`,0.1`,0.05`
7,D1STR`,2`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,1`,18.333333333333332`,1.0`,0.1`,0.05`
8,D2STR`,1`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,2`,-15.0`,0.5`,0.005`,0.05`
9,D2STR`,2`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,2`,-15.0`,0.5`,0.005`,0.05`


In [134]:
def CreateSynapses(popdata, cons, effs, plasticity):
    connection = np.zeros((len(popdata),len(popdata))).tolist()
    efficacy = np.zeros((len(popdata),len(popdata))).tolist()

    for idx1,row1 in popdata.iterrows():
        for idx2,row2 in popdata.iterrows():
            con = cons.iloc[idx1,idx2]

            if con == 0.0:
                efficacy[idx1][idx2] = None
                connection[idx1][idx2] = None
                continue

            eff = effs.iloc[idx1,idx2]
            
            condata = None
            if con == 1.0:
                if plasticity.iloc[idx1][idx2]:
                    condata = np.ones((popdata['N'].loc[idx1],popdata['N'].loc[idx2]))
                else:
                    condata = np.ones((popdata['N'].loc[idx1],1))
            else:
                condata = (np.random.rand(popdata['N'].loc[idx1],popdata['N'].loc[idx2]) < con).astype(int)
            
            effdata = condata * untrace(eff)

            connection[idx1][idx2] = condata
            efficacy[idx1][idx2] = effdata

            print(idx1,idx2,con)
    return connection,efficacy

In [135]:
connectivity_AMPA

,GPi,GPi,STNE,STNE,GPeP,GPeP,D1STR,D1STR,D2STR,D2STR,LIP,LIP,Th,Th,FSI,LIPI
GPi,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
GPi,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
STNE,0.0`,0.0`,0.0`,0.0`,0.161668`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
STNE,0.0`,0.0`,0.0`,0.0`,0.0`,0.161668`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
GPeP,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
GPeP,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
D1STR,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
D1STR,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
D2STR,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
D2STR,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`


In [136]:
connectivity_GABA

,GPi,GPi,STNE,STNE,GPeP,GPeP,D1STR,D1STR,D2STR,D2STR,LIP,LIP,Th,Th,FSI,LIPI
GPi,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,1.0`,0.0`,0.0`,0.0`
GPi,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,1.0`,0.0`,0.0`
STNE,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
STNE,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
GPeP,1.0`,0.0`,0.0667`,0.0`,0.0667`,0.0667`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
GPeP,0.0`,1.0`,0.0`,0.0667`,0.0667`,0.0667`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
D1STR,1.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.45`,0.0`,0.45`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
D1STR,0.0`,1.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.45`,0.0`,0.45`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
D2STR,0.0`,0.0`,0.0`,0.0`,1.0`,0.0`,0.5`,0.0`,0.45`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`
D2STR,0.0`,0.0`,0.0`,0.0`,0.0`,1.0`,0.0`,0.5`,0.0`,0.45`,0.0`,0.0`,0.0`,0.0`,0.0`,0.0`


In [137]:
def CreateAuxiliarySynapseData(popdata, cons):
    auxdata = np.zeros((len(popdata),len(popdata))).tolist()
    for idx1,row1 in popdata.iterrows():
        for idx2,row2 in popdata.iterrows():
            if cons.iloc[idx1,idx2] == 0.0:
                auxdata[idx1][idx2] = None
                continue
            auxdata[idx1][idx2] = np.ones((popdata['N'].loc[idx1],popdata['N'].loc[idx2]))
            # print(idx1,idx2)
    return auxdata

In [138]:
def expandParamByCell(popdata,param,defaultvalue=np.nan):
    
    databypop = []
    
    
    if param not in popdata.columns:
        #print(param + " not found, initializing to " + str(defaultvalue))
        pass
    
    for idx1,row1 in popdata.iterrows():
        
        fillvalue = defaultvalue
        if param in popdata.columns:
            if not row1[param].is_nan():
                fillvalue = untrace(row1[param])
        
        
        array = np.ones(row1['N']) * fillvalue
        databypop.append(array)
    
    return databypop

In [139]:
def expandParamByCell2D(popdata,param,fillvalue=np.nan):
    
    databypop = []
    
    
    if param not in popdata.columns:
        print(param + " not found, initializing to " + str(fillvalue))
    
    for idx1,row1 in popdata.iterrows():
        
        if param not in popdata.columns:
            array = np.ones(row1['N']) * fillvalue
            databypop.append(array)
            continue

        
        value = row1[param]
        if value.is_nan():
            value = fillvalue
        else:
            value = untrace(value)
        array = [value]*untrace(row1['N'])
        databypop.append(np.array(array))
        continue
    
    return databypop

In [140]:
class NullClass():
    pass

In [141]:
def initializeAgent(popdata):
    agent = NullClass()
    
    propertylist = [
        'FreqExt_AMPA',
        'FreqExt_GABA',
        'FreqExt_NMDA',
        'MeanExtEff_AMPA',
        'MeanExtEff_GABA',
        'MeanExtEff_NMDA',
        'MeanExtCon_AMPA',
        'MeanExtCon_GABA',
        'MeanExtCon_NMDA',
        'Tau_AMPA',
        'Tau_GABA',
        'Tau_NMDA',
        'ExtS_AMPA',
        'ExtS_GABA',
        'ExtS_NMDA',
        'LS_AMPA',
        'LS_GABA',
        'LS_NMDA',
        'timesincelastspike',
        'g_rb',
        'V',
        'V_h',
        'h',
        'tauhp',
        'tauhm',
        'g_T',
        'Threshold',
        'ResetPot',
        'RefrState',
        'g_adr_max',
        'Vadr_h',
        'Vadr_s',
        'g_k',
        'g_k_max',
        'tau_k_max',
        'Vk_h',
        'Vk_s',
        'n_k',
        'g_ahp',
        'Taum',
        'RestPot',
        'C',
        'ADRRevPot',
        'V_T',
        'Ca',
        'Vk',
        'Tau_ca',
        'RevPot_AMPA',
        'RevPot_GABA',
        'RevPot_NMDA',
        'alpha_ca',
        'dpmn_DAp',
        'dpmn_tauDOP',
        'dpmn_APRE',
        'dpmn_dPRE',
        'dpmn_tauPRE',
        'dpmn_APOST',
        'dpmn_dPOST',
        'dpmn_tauPOST',
        'dpmn_E',
        'dpmn_tauE',
        'dpmn_m',
        'dpmn_DAt',
        'dpmn_taum',
        'dpmn_type',
        'dpmn_wmax',
        'dpmn_alphaw',
        ###
        'ExtMuS_AMPA',
        'ExtMuS_GABA',
        'ExtMuS_NMDA',
        'ExtSigmaS_AMPA',
        'ExtSigmaS_GABA',
        'ExtSigmaS_NMDA',
        'cond',
        'g_adr',
        'dv',
        'tau_n',
        'n_inif',
        'Vaux',
        'N'
    ]
    
    for prop in propertylist:
        setattr(agent, prop, expandParamByCell(popdata,prop,0))
        
    agent.LastConductanceNMDA = CreateAuxiliarySynapseData(popdata,connectivity_NMDA)
    
    agent.spikes = []
    for i in range(len(popdata)):
        agent.spikes.append([])
        
    agent.dt = 0.2 # ms
    
    agent.bufferlength = 50 # 10ms averaging window / 0.2ms dt
    agent.bufferpointer = 0
    agent.rollingbuffer = np.zeros((len(popdata),agent.bufferlength))
    
    return agent

In [ ]:
def codeblock_init_agent(self):
    self.AMPA_con,self.AMPA_eff = CreateSynapses(self.popdata,self.connectivity_AMPA,self.meaneff_AMPA,self.plastic_AMPA)
    self.GABA_con,self.GABA_eff = CreateSynapses(self.popdata,self.connectivity_GABA,self.meaneff_GABA,self.plastic_GABA)
    self.NMDA_con,self.NMDA_eff = CreateSynapses(self.popdata,self.connectivity_NMDA,self.meaneff_NMDA,self.plastic_NMDA)
    
    popdata = self.popdata
    agent = initializeAgent(popdata)
    self.agent = agent
    
    agent.AMPA_con,agent.AMPA_eff = self.AMPA_con,self.AMPA_eff
    agent.GABA_con,agent.GABA_eff = self.GABA_con,self.GABA_eff
    agent.NMDA_con,agent.NMDA_eff = self.NMDA_con,self.NMDA_eff

In [39]:
AMPA_con,AMPA_eff = CreateSynapses(popdata,connectivity_AMPA,meaneff_AMPA,plastic_AMPA)
GABA_con,GABA_eff = CreateSynapses(popdata,connectivity_GABA,meaneff_GABA,plastic_GABA)
NMDA_con,NMDA_eff = CreateSynapses(popdata,connectivity_NMDA,meaneff_NMDA,plastic_NMDA)

2 4 0.161668`
3 5 0.161668`
10 6 1.0`
10 8 1.0`
10 10 0.4335`
10 11 0.4335`
10 12 1.0`
10 13 1.0`
10 14 1.0`
10 15 0.241667`
11 7 1.0`
11 9 1.0`
11 10 0.4335`
11 11 0.4335`
11 12 1.0`
11 13 1.0`
11 14 1.0`
11 15 0.241667`
12 6 1.0`
12 8 1.0`
12 14 0.8334`
13 7 1.0`
13 9 1.0`
13 14 0.8334`
0 12 1.0`
1 13 1.0`
4 0 1.0`
4 2 0.0667`
4 4 0.0667`
4 5 0.0667`
5 1 1.0`
5 3 0.0667`
5 4 0.0667`
5 5 0.0667`
6 0 1.0`
6 6 0.45`
6 8 0.45`
7 1 1.0`
7 7 0.45`
7 9 0.45`
8 4 1.0`
8 6 0.5`
8 8 0.45`
9 5 1.0`
9 7 0.5`
9 9 0.45`
14 6 1.0`
14 7 1.0`
14 8 1.0`
14 9 1.0`
14 14 1.0`
15 10 1.0`
15 11 1.0`
15 15 1.0`
2 0 1.0`
2 1 1.0`
2 4 0.161668`
3 0 1.0`
3 1 1.0`
3 5 0.161668`
10 6 1.0`
10 8 1.0`
10 10 0.4335`
10 11 0.4335`
10 12 1.0`
10 13 1.0`
10 15 0.241667`
11 7 1.0`
11 9 1.0`
11 10 0.4335`
11 11 0.4335`
11 12 1.0`
11 13 1.0`
11 15 0.241667`
12 10 0.8334`
12 11 0.8334`
12 15 0.8334`
13 10 0.8334`
13 11 0.8334`
13 15 0.8334`


In [142]:
agent = initializeAgent(popdata)

In [143]:
agent.AMPA_con,agent.AMPA_eff = AMPA_con,AMPA_eff
agent.GABA_con,agent.GABA_eff = GABA_con,GABA_eff
agent.NMDA_con,agent.NMDA_eff = NMDA_con,NMDA_eff

In [148]:
agent.spikes[0].append(1)
agent.spikes[2].append(1)
agent.spikes[11].append(1)

In [149]:
multitimestep_mutator(agent,popdata,1000)

In [150]:
agent.rollingbuffer

array([[  0.,  75.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,  75.,   0.,
          0.,   0.,  75.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,  75.,
          0.,   0.,   0.,  75.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,
         75.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,  75.,   0.,   0.,
          0.,  75.,   0.,   0.,   0.,  75.],
       [ 75.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,  75.,   0.,   0.,
          0.,  75.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,  75.,   0.,
          0.,   0.,  75.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,  75.,
          0.,   0.,   0.,  75.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,
         75.,   0.,   0.,   0.,  75.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   

In [ ]:
other agent properties
phase = 0 (ready to decide), 1 (motor delay), 2 (dpmn delay)
phasetimer = ms since phase start
motor_queued
dpmn_queued

gain = array
extstim = array
ramping_extstim = array




In [152]:
agent.phase = 0
agent.phasetimer = 0
agent.motor_queued = None
agent.dpmn_queued = None
agent.gain = np.ones(len(actionchannels))
agent.extstim = np.zeros(len(actionchannels))
agent.ramping_extstim = np.zeros(len(actionchannels))
agent.in_popids = np.where(popdata['name'] == 'LIP')[0]
agent.out_popids = np.where(popdata['name'] == 'Th')[0]

In [153]:
agent.gain

array([1., 1.])

In [154]:
np.where(popdata['name'] == 'LIP')[0]

array([10, 11], dtype=int64)

In [155]:
agent.in_popids

array([10, 11], dtype=int64)

In [181]:
agent.out_popids

array([12, 13], dtype=int64)

In [156]:
presented_stimulus = 1

In [183]:
# phase = 0

agent.phasetimer += 1
agent.motor_queued = None
agent.dpmn_queued = None
agent.extstim = agent.gain * presented_stimulus * 2.5 #*1000 # TODO: make 2.5 a param
agent.ramping_extstim = agent.ramping_extstim * 0.9 + agent.extstim * 0.1
agent.ramping_extstim

array([250.225, 250.225])

In [184]:
for action_idx in range(len(actionchannels)):
    popid = agent.in_popids[action_idx]
    agent.FreqExt_AMPA[popid] = np.ones(len(agent.FreqExt_AMPA[popid])) * agent.ramping_extstim[action_idx]

In [ ]:
print(agent.FreqExt_AMPA[popid])
print(agent.ExtS_AMPA[popid])
print(agent.V[popid])

In [202]:
gateFRs = agent.rollingbuffer[agent.out_popids].mean(1) / untrace(list(popdata['N'][agent.out_popids])) / agent.dt * 1000
gateFRs

array([2500., 2500.])

In [189]:
popdata['N']

0      75`
1      75`
2     750`
3     750`
4     750`
5     750`
6      75`
7      75`
8      75`
9      75`
10    204`
11    204`
12     75`
13     75`
14     75`
15    186`
Name: N, dtype: object

In [185]:
multitimestep_mutator(agent,popdata,100)
agent.rollingbuffer

array([[  0.,   0.,   0.,  75.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,  75.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,  75.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         75.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  75.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  75.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,  75.],
       [  5.,   7.,  11.,   7.,   5.,   9.,   9.,   6.,   9.,   8.,  11.,
          7.,   7.,   6.,  11.,  10.,   7.,   7.,  11.,   6.,   9.,   3.,
          6.,  12.,  10.,   8.,   4.,   5.,  13.,   7.,   4.,  10.,   9.,
          8.,   7.,   7.,  11.,   7.,   5.,   8.,  11.,   6.,   6.,  12.,
          9.,   5.,   

In [206]:
thresholds_crossed = np.where(gateFRs > 30)[0]


array([0, 1], dtype=int64)